In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-03 18:47:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.73MB/s    in 0.2s    

2022-04-03 18:47:50 (5.73 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [6]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

greaterthan20_df = df.filter("total_votes >= 20")
greaterthan20_df.show()






+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   11025729| ROP6ITXO8K5V2|B00LE9MZMW|     778942419|YJ Moyu 13x13x13 ...|            Toys|          5|           23|         27|   N|                Y|          Five Stars|Great cube, moves...| 2015-08-31|
|         US|   50123725|R3ND1LVU7AXCVF|B00LOV24VC|     363255349|Marshmallow Furni...|            Toys|          1|    

In [9]:
print(greaterthan20_df.count())


70474


In [8]:
#Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%. 
helpfulgreaterthan50_df = greaterthan20_df.filter("helpful_votes/total_votes>=.50")
helpfulgreaterthan50_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   11025729| ROP6ITXO8K5V2|B00LE9MZMW|     778942419|YJ Moyu 13x13x13 ...|            Toys|          5|           23|         27|   N|                Y|          Five Stars|Great cube, moves...| 2015-08-31|
|         US|   50123725|R3ND1LVU7AXCVF|B00LOV24VC|     363255349|Marshmallow Furni...|            Toys|          1|    

In [10]:
print(helpfulgreaterthan50_df.count())

63294


In [11]:
#Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paidreview_df = helpfulgreaterthan50_df.filter(helpfulgreaterthan50_df["vine"]=="Y")
paidreview_df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53067113| R9I5FOLKU99RY|B00U1KA12G|     278300201|Qixels S1 Turbo D...|            Toys|          5|           19|         20|   Y|                N|11 Year Old Loves...|My daughter who i...| 2015-08-31|
|         US|   44759540|R3ED60RC69CCQ6|B00VJKTLMS|     917726757|Fisher-Price Nick...|            Toys|          5|    

In [12]:
print(paidreview_df.count())

1266


In [13]:
#Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
notpaidreview_df = helpfulgreaterthan50_df.filter(helpfulgreaterthan50_df["vine"]=="N")
notpaidreview_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   11025729| ROP6ITXO8K5V2|B00LE9MZMW|     778942419|YJ Moyu 13x13x13 ...|            Toys|          5|           23|         27|   N|                Y|          Five Stars|Great cube, moves...| 2015-08-31|
|         US|   50123725|R3ND1LVU7AXCVF|B00LOV24VC|     363255349|Marshmallow Furni...|            Toys|          1|    

In [14]:
print(notpaidreview_df.count())

62028


In [16]:
#Determine the total number of reviews

print(df.count())





4864249


In [17]:
 # determine the number of 5-star reviews

fivestarreview_df = df.filter(df["star_rating"]=="5")
fivestarreview_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [18]:
#count of five star reviews


print(fivestarreview_df.count())

3076917


In [19]:
# create unpaid entirepopulation dataframe
entirenotpaidreview_df = df.filter(df["vine"]=="N")
entirenotpaidreview_df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [20]:
print(entirenotpaidreview_df.count())

4822408


In [21]:
# create paid entirepopulation dataframe
entirepaidreview_df = df.filter(df["vine"]=="Y")
entirepaidreview_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   27225859| R4R337CCDWLNG|B00YRA3H4U|     223420727|Franklin Sports M...|            Toys|          3|            0|          1|   Y|                N|Got wrong product...|Got a wrong produ...| 2015-08-31|
|         US|   27367902|R16N0A77B9KDEJ|B00WJ1PPGM|     999209932|LeapFrog LeapTV: ...|            Toys|          4|    

In [22]:
print(entirepaidreview_df.count())

41835


In [25]:
#  determine the numberof 5-star reviews for the unpaid reviews

unpaidfivestarreview_df = entirenotpaidreview_df.filter(entirenotpaidreview_df["star_rating"]=="5")
unpaidfivestarreview_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [27]:
print(unpaidfivestarreview_df.count())

3059846


In [26]:
#  determine the numberof 5-star reviews for the paid reviews


paidfivestarreview_df = entirepaidreview_df.filter(entirepaidreview_df["star_rating"]=="5")
paidfivestarreview_df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43450674|R3TQ9HOI4SGLKE|B00C1ZX298|     238990333|Madame Alexander ...|            Toys|          5|            0|          0|   Y|                N|Super Cute! Equal...|Really cute doll,...| 2015-08-31|
|         US|   51118641|R3V2Y9NU2GKN0M|B00Y53UVIO|     689858218|VTech Magic Star ...|            Toys|          5|    

In [24]:
print(paidfivestarreview_df.count())

17071


In [37]:
#percentage of 5 star reviews for nonvine reviews

five_star_percentage_unpaid = ((unpaidfivestarreview_df.count()) / (entirenotpaidreview_df.count()))

print(five_star_percentage_unpaid)


0.6345058319412211


In [38]:
#percentage of 5 star reviews for vine reviews

five_star_percentage_paid = ((paidfivestarreview_df.count()) / (entirepaidreview_df.count()))

print(five_star_percentage_paid)

0.40805545595792997
